In [2]:
# using ast.literal_eval()
import sys
import ast
import os
import time
import pandas as pd
import pickle
import glob
import numpy as np
import statistics
from scipy.stats import mannwhitneyu
import math

In [3]:
with open("hPPIN_paths.dictionary", 'rb') as config_dictionary_file:
 
    # Step 3
    paths = pickle.load(config_dictionary_file)

In [4]:
# create list
file2 = "common_nodes_hPPIN_STRING_PANCAN.txt"
combined_list = []
with open(file2) as IN:
    for line in IN:
        combined_list.append(line.rstrip().split("\t")[0])

In [5]:
def compute_sp(list1, list2, paths):
    total_sp = []
    for i in range(len(list1)):
        for j in range(len(list2)):
            if(list1[i] in paths.keys()):
                shortest_paths = paths[list1[i]]
                if(list2[j] in shortest_paths.keys()):
                    total_sp.append(shortest_paths[list2[j]])
    return({"distribution": total_sp,
            "mean": statistics.mean(total_sp),
            "median": statistics.median(total_sp)})

In [6]:
# Read Cancersea Data
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/hallmark"

filenames = os.listdir(folder_path)

hallmark = {}

for filename in filenames:
    file_path = f"{folder_path}/{filename}"
    hallmark[filename[:-4]] = list()

    with open(file_path) as IN:
        for line in IN:
            hallmark[filename[:-4]].append(line.rstrip())

In [7]:
hallmark["COSMIC"] = list()
with open("/data/timonaj/cancer_as_wound/ppi_analysis/cosmic_hallmarks.txt") as IN:
    for line in IN:
        hallmark["COSMIC"].append(line.rstrip().split("\t")[0])

In [8]:
# read in HP sets
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets/"

filenames = os.listdir(folder_path)

hp_genesets = {}

for filename in filenames:
    file_path = f"{folder_path}/{filename}"
    hp_genesets[filename[:-4]] = list()

    with open(file_path) as IN:
        for line in IN:
            hp_genesets[filename[:-4]].append(line.rstrip().split("\t")[0])

In [9]:
# read in HP matched controls set
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets_matched_controls/"

filenames = glob.glob("/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets_matched_controls/hPPIN*")
hp_genesets_matched = {}

for filename in filenames:
    name = filename.split("hPPIN_")[1][:-4]
    hp_genesets_matched[name] = list()

    with open(filename) as IN:
        for line in IN:
            hp_genesets_matched[name].append(line.rstrip().split("\t")[0])

In [10]:
a_mean = np.zeros(shape=(len(hp_genesets), len(hallmark)))
a_median = np.zeros(shape=(len(hp_genesets), len(hallmark)))
a_dist = pd.DataFrame(index = range(len(hallmark)),
                      columns = range(len(hp_genesets)))

In [11]:
i=0
for key1,value1 in hp_genesets.items():
    j=0
    for key2,value2 in hallmark.items():
        current_info = compute_sp(value1, value2, paths)
        a_mean[i][j] = current_info["mean"]
        a_median[i][j] = current_info["median"]
        a_dist[i][j] = current_info["distribution"]
        j+=1
    i+=1

In [12]:
pd.DataFrame(a_median)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
4,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
5,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
6,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
7,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
8,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [13]:
pd.DataFrame(a_mean)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,3.015029,2.990722,2.787826,2.898138,2.879450,2.908349,2.742093,2.878060,2.777083,3.045388,...,2.952940,3.022683,2.926624,2.955483,2.771729,2.649483,2.983630,2.943429,2.947333,2.825073
1,3.144162,3.139969,2.890764,3.051414,2.991639,3.017597,2.937205,2.970459,2.885564,3.127025,...,3.068110,3.119592,3.021875,3.134606,2.884982,2.795574,3.108485,3.103048,3.052092,2.950114
2,3.115585,3.084097,2.865177,3.002217,2.977958,3.005455,2.873022,2.969875,2.857645,3.143381,...,3.047505,3.123524,3.014244,3.076084,2.865655,2.750266,3.079021,3.026549,3.047738,2.916098
3,2.961315,2.874008,2.787091,2.805283,2.860668,2.864856,2.661346,2.890655,2.765544,3.006007,...,2.904041,2.995726,2.919366,2.871786,2.773071,2.568127,2.934892,2.819566,2.926636,2.772855
4,2.947844,2.898562,2.717136,2.818732,2.813612,2.828167,2.673359,2.821507,2.700771,2.972382,...,2.874117,2.951106,2.860017,2.885026,2.708241,2.561027,2.903339,2.839225,2.887314,2.743050
5,3.087146,3.071907,2.849971,2.982147,2.953303,2.980851,2.846840,2.944214,2.839853,3.112674,...,3.024967,3.094647,2.991211,3.053044,2.838936,2.731459,3.056493,3.028442,3.016739,2.898832
6,3.052044,3.041483,2.822198,2.950665,2.911392,2.943253,2.817192,2.907839,2.810454,3.077507,...,2.988386,3.052978,2.956935,3.019547,2.802375,2.693725,3.024686,2.998724,2.983949,2.863167
7,2.910627,2.932939,2.647195,2.819354,2.752309,2.804938,2.672959,2.760881,2.636849,2.960017,...,2.849721,2.924601,2.799268,2.884069,2.630465,2.530444,2.885181,2.881454,2.851810,2.711501
8,3.177504,3.191453,2.941501,3.095639,3.020481,3.061481,2.987817,2.997676,2.936294,3.170135,...,3.113950,3.156884,3.060505,3.178530,2.913853,2.836731,3.152729,3.157087,3.094390,2.996696


In [14]:
pd.DataFrame(a_mean, columns = hallmark.keys(), index = hp_genesets.keys()).to_csv("hPPIN_meanSP_HP_Phenotypes_hallmark.csv")

In [15]:
a_matched_mean = np.zeros(shape=(len(hp_genesets_matched), len(hallmark)))
a_matched_median = np.zeros(shape=(len(hp_genesets_matched), len(hallmark)))
a_matched_dist = pd.DataFrame(index = range(len(hallmark)),
                              columns = range(len(hp_genesets_matched)))

In [16]:
i=0
for key1,value1 in hp_genesets_matched.items():
    j=0
    for key2,value2 in hallmark.items():
        current_info = compute_sp(value1, value2, paths)
        a_matched_mean[i][j] = current_info["mean"]
        a_matched_median[i][j] = current_info["median"]
        a_matched_dist[i][j] = current_info["distribution"]
        j+=1
    i+=1

In [17]:
pd.DataFrame(a_matched_median)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
4,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
5,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
6,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
7,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
8,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [18]:
pd.DataFrame(a_matched_mean)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,3.142523,3.184640,2.889379,3.073822,2.990680,3.046007,2.956831,2.960849,2.887208,3.195108,...,3.093677,3.157711,3.012442,3.143373,2.874664,2.801604,3.128430,3.142101,3.084566,2.962097
1,3.147152,3.189600,2.901366,3.080598,2.995109,3.051160,2.962195,2.968305,2.898268,3.198869,...,3.100342,3.163365,3.021599,3.147124,2.880938,2.813657,3.135910,3.149373,3.091780,2.969781
2,3.302109,3.345925,3.064789,3.235460,3.153915,3.209332,3.133429,3.132395,3.065520,3.351130,...,3.259565,3.319770,3.186015,3.317648,3.050296,2.984167,3.294593,3.292218,3.244853,3.127144
3,3.151961,3.195378,2.906454,3.086972,3.001509,3.055820,2.967972,2.971295,2.903623,3.199811,...,3.103221,3.164592,3.025558,3.158560,2.888486,2.817041,3.143146,3.153918,3.088800,2.971958
4,3.506400,3.549550,3.254812,3.438030,3.349458,3.407113,3.335227,3.334938,3.257073,3.556708,...,3.467625,3.522757,3.366640,3.529232,3.228674,3.190341,3.500563,3.506006,3.439050,3.321094
5,3.194575,3.236366,2.950920,3.126836,3.044790,3.097101,3.012462,3.021262,2.947174,3.247366,...,3.147201,3.209973,3.068587,3.195875,2.935643,2.857905,3.184356,3.188840,3.134817,3.015248
6,3.174526,3.212261,2.924550,3.103529,3.015974,3.076607,2.984723,2.998521,2.923549,3.219058,...,3.124764,3.182587,3.044298,3.174679,2.907873,2.839744,3.162751,3.171059,3.109326,2.995577
7,3.134008,3.183787,2.897920,3.071558,2.984863,3.045419,2.959084,2.967675,2.893702,3.192925,...,3.094417,3.154241,3.022955,3.144083,2.878883,2.796800,3.130467,3.135985,3.084183,2.961085
8,3.155873,3.201666,2.903293,3.093170,3.003076,3.059330,2.983983,2.974983,2.903124,3.203925,...,3.110787,3.166885,3.024434,3.167228,2.883801,2.826320,3.148319,3.158717,3.094556,2.980145


In [19]:
pd.DataFrame(a_matched_mean, columns = hallmark.keys(), index = hp_genesets_matched.keys()).to_csv("hPPIN_meanSP_HPmatched_Phenotypes_hallmark.csv")

In [20]:
mannwhitneyu_p = np.zeros(shape=(len(hp_genesets), len(hallmark)))
test_dist = np.zeros(shape=(len(hp_genesets), len(hallmark)))
test_matched_dist = np.zeros(shape=(len(hp_genesets), len(hallmark)))

In [21]:
LogFC = np.zeros(shape=(len(hp_genesets), len(hallmark)))
for i in range(len(hp_genesets)):
    for j in range(len(hallmark)):
        actual_dist = a_dist[i][j]
        control_dist = a_matched_dist[i][j]
        res = mannwhitneyu(actual_dist, control_dist, alternative="less")
        mannwhitneyu_p[i][j] = res.pvalue
        test_dist[i][j] = statistics.mean(actual_dist)
        test_matched_dist[i][j] = statistics.mean(control_dist)
        LogFC[i][j] = math.log((test_dist[i][j]/test_matched_dist[i][j]))

In [22]:
hallmark.keys()

dict_keys(['PROTEIN_SECRETION', 'COAGULATION', 'E2F_TARGETS', 'COMPLEMENT', 'UNFOLDED_PROTEIN_RESPONSE', 'UV_RESPONSE_UP', 'IL6_JAK_STAT3_SIGNALING', 'MYC_TARGETS_V2', 'G2M_CHECKPOINT', 'XENOBIOTIC_METABOLISM', 'ESTROGEN_RESPONSE_EARLY', 'HEME_METABOLISM', 'WNT_BETA_CATENIN_SIGNALING', 'APOPTOSIS', 'KRAS_SIGNALING_UP', 'APICAL_SURFACE', 'NOTCH_SIGNALING', 'TNFA_SIGNALING_VIA_NFKB', 'GLYCOLYSIS', 'APICAL_JUNCTION', 'IL2_STAT5_SIGNALING', 'P53_PATHWAY', 'PANCREAS_BETA_CELLS', 'BILE_ACID_METABOLISM', 'INTERFERON_ALPHA_RESPONSE', 'HEDGEHOG_SIGNALING', 'ALLOGRAFT_REJECTION', 'MITOTIC_SPINDLE', 'TGF_BETA_SIGNALING', 'UV_RESPONSE_DN', 'KRAS_SIGNALING_DN', 'PEROXISOME', 'FATTY_ACID_METABOLISM', 'EPITHELIAL_MESENCHYMAL_TRANSITION', 'SPERMATOGENESIS', 'MTORC1_SIGNALING', 'CHOLESTEROL_HOMEOSTASIS', 'OXIDATIVE_PHOSPHORYLATION', 'MYOGENESIS', 'INTERFERON_GAMMA_RESPONSE', 'HYPOXIA', 'ANDROGEN_RESPONSE', 'ADIPOGENESIS', 'DNA_REPAIR', 'INFLAMMATORY_RESPONSE', 'MYC_TARGETS_V1', 'PI3K_AKT_MTOR_SIGNALING

In [23]:
pvals = pd.DataFrame(mannwhitneyu_p, columns = hallmark.keys(), index = hp_genesets.keys())

In [24]:
pvals

,PROTEIN_SECRETION,COAGULATION,E2F_TARGETS,COMPLEMENT,UNFOLDED_PROTEIN_RESPONSE,UV_RESPONSE_UP,IL6_JAK_STAT3_SIGNALING,MYC_TARGETS_V2,G2M_CHECKPOINT,XENOBIOTIC_METABOLISM,...,ANDROGEN_RESPONSE,ADIPOGENESIS,DNA_REPAIR,INFLAMMATORY_RESPONSE,MYC_TARGETS_V1,PI3K_AKT_MTOR_SIGNALING,ESTROGEN_RESPONSE_LATE,ANGIOGENESIS,REACTIVE_OXYGEN_SPECIES_PATHWAY,COSMIC
downregulated.wound,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.727956e-304,0.000000e+00,0.000000e+00,2.672554e-82,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,8.188465e-244,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.465023e-300,1.455994e-245,0.000000e+00
upregulated.stress,9.971601e-01,1.973300e-47,3.917671e-01,7.063001e-20,9.826904e-01,9.053126e-22,2.316964e-07,9.989936e-01,2.154432e-01,6.739451e-144,...,4.656598e-12,2.168655e-43,9.971699e-01,0.141475,1.000000e+00,1.168957e-02,6.572692e-15,1.219086e-10,2.301601e-11,1.171874e-19
downregulated.regen,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.588313e-221,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,7.245809e-303,4.081946e-294,0.000000e+00
msigdb.wound,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.644638e-61,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,1.405621e-276,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.271941e-229,0.000000e+00
msigdb.regen,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
upregulated.regen,4.620868e-145,0.000000e+00,1.882372e-289,0.000000e+00,1.199597e-118,0.000000e+00,0.000000e+00,7.381329e-45,6.001427e-302,0.000000e+00,...,1.233153e-194,0.000000e+00,2.703809e-120,0.000000,1.936318e-310,4.907018e-281,0.000000e+00,1.178590e-119,2.916611e-108,0.000000e+00
upregulated.wound,4.798841e-169,0.000000e+00,7.746919e-243,0.000000e+00,1.805032e-130,0.000000e+00,0.000000e+00,4.547260e-51,6.717871e-293,0.000000e+00,...,8.180282e-213,0.000000e+00,2.691546e-130,0.000000,5.831895e-298,0.000000e+00,0.000000e+00,1.777370e-126,1.353017e-102,0.000000e+00
msigdb.stress,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
downregulated.stress,1.000000e+00,4.713750e-01,1.000000e+00,1.000000e+00,1.000000e+00,9.999998e-01,9.999982e-01,1.000000e+00,1.000000e+00,9.857803e-19,...,9.999007e-01,6.424374e-01,1.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,8.954091e-01,9.949841e-01,1.000000e+00


In [25]:
pvals.to_csv("hPPIN_matched_pvals_hallmark.csv")

In [26]:
LFC = pd.DataFrame(LogFC, columns = hallmark.keys(), index = hp_genesets.keys())

In [27]:
LFC

,PROTEIN_SECRETION,COAGULATION,E2F_TARGETS,COMPLEMENT,UNFOLDED_PROTEIN_RESPONSE,UV_RESPONSE_UP,IL6_JAK_STAT3_SIGNALING,MYC_TARGETS_V2,G2M_CHECKPOINT,XENOBIOTIC_METABOLISM,...,ANDROGEN_RESPONSE,ADIPOGENESIS,DNA_REPAIR,INFLAMMATORY_RESPONSE,MYC_TARGETS_V1,PI3K_AKT_MTOR_SIGNALING,ESTROGEN_RESPONSE_LATE,ANGIOGENESIS,REACTIVE_OXYGEN_SPECIES_PATHWAY,COSMIC
downregulated.wound,-0.041416,-0.062825,-0.035780,-0.058853,-0.037902,-0.046246,-0.075397,-0.028360,-0.038889,-0.047992,...,-0.046559,-0.043703,-0.028901,-0.061634,-0.036465,-0.055828,-0.047391,-0.065317,-0.045510,-0.047363
upregulated.stress,-0.000951,-0.015683,-0.003661,-0.009519,-0.001159,-0.011061,-0.008472,0.000725,-0.004393,-0.022715,...,-0.010451,-0.013934,0.000091,-0.003986,0.001402,-0.006448,-0.008784,-0.014818,-0.012920,-0.006644
downregulated.regen,-0.058144,-0.081484,-0.067349,-0.074820,-0.057406,-0.065634,-0.086764,-0.053278,-0.070219,-0.063998,...,-0.067271,-0.060933,-0.055422,-0.075599,-0.062442,-0.081623,-0.067671,-0.084138,-0.062671,-0.069874
msigdb.wound,-0.062392,-0.105998,-0.041936,-0.095686,-0.048060,-0.064530,-0.109047,-0.027515,-0.048722,-0.062479,...,-0.066337,-0.054837,-0.035729,-0.095182,-0.040777,-0.092510,-0.068553,-0.112062,-0.053929,-0.069344
msigdb.regen,-0.173516,-0.202606,-0.180556,-0.198612,-0.174330,-0.186236,-0.221205,-0.167183,-0.187292,-0.179472,...,-0.187724,-0.177064,-0.163088,-0.201547,-0.175772,-0.219720,-0.187062,-0.210946,-0.174869,-0.191224
upregulated.regen,-0.034207,-0.052152,-0.034808,-0.047378,-0.030508,-0.038258,-0.056548,-0.025833,-0.037095,-0.042362,...,-0.039613,-0.036589,-0.025539,-0.045722,-0.033497,-0.045253,-0.040982,-0.051609,-0.038394,-0.039374
upregulated.wound,-0.039347,-0.054630,-0.035625,-0.050509,-0.035291,-0.044312,-0.057766,-0.030709,-0.039452,-0.044969,...,-0.044625,-0.041577,-0.029117,-0.050100,-0.036954,-0.052789,-0.044635,-0.055879,-0.041158,-0.045209
msigdb.stress,-0.073944,-0.082066,-0.090493,-0.085677,-0.081114,-0.082257,-0.101694,-0.072229,-0.092952,-0.075742,...,-0.082378,-0.075590,-0.076877,-0.086320,-0.090242,-0.100081,-0.081595,-0.084648,-0.078333,-0.088053
downregulated.stress,0.006831,-0.003195,0.013074,0.000798,0.005779,0.000703,0.001284,0.007599,0.011361,-0.010602,...,0.001016,-0.003163,0.011856,0.003562,0.010367,0.003677,0.001400,-0.000516,-0.000053,0.005539


In [28]:
LFC.to_csv("hPPIN_matched_LFC_hallmark.csv")

In [29]:
pd.DataFrame(test_dist)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,3.015029,2.990722,2.787826,2.898138,2.879450,2.908349,2.742093,2.878060,2.777083,3.045388,...,2.952940,3.022683,2.926624,2.955483,2.771729,2.649483,2.983630,2.943429,2.947333,2.825073
1,3.144162,3.139969,2.890764,3.051414,2.991639,3.017597,2.937205,2.970459,2.885564,3.127025,...,3.068110,3.119592,3.021875,3.134606,2.884982,2.795574,3.108485,3.103048,3.052092,2.950114
2,3.115585,3.084097,2.865177,3.002217,2.977958,3.005455,2.873022,2.969875,2.857645,3.143381,...,3.047505,3.123524,3.014244,3.076084,2.865655,2.750266,3.079021,3.026549,3.047738,2.916098
3,2.961315,2.874008,2.787091,2.805283,2.860668,2.864856,2.661346,2.890655,2.765544,3.006007,...,2.904041,2.995726,2.919366,2.871786,2.773071,2.568127,2.934892,2.819566,2.926636,2.772855
4,2.947844,2.898562,2.717136,2.818732,2.813612,2.828167,2.673359,2.821507,2.700771,2.972382,...,2.874117,2.951106,2.860017,2.885026,2.708241,2.561027,2.903339,2.839225,2.887314,2.743050
5,3.087146,3.071907,2.849971,2.982147,2.953303,2.980851,2.846840,2.944214,2.839853,3.112674,...,3.024967,3.094647,2.991211,3.053044,2.838936,2.731459,3.056493,3.028442,3.016739,2.898832
6,3.052044,3.041483,2.822198,2.950665,2.911392,2.943253,2.817192,2.907839,2.810454,3.077507,...,2.988386,3.052978,2.956935,3.019547,2.802375,2.693725,3.024686,2.998724,2.983949,2.863167
7,2.910627,2.932939,2.647195,2.819354,2.752309,2.804938,2.672959,2.760881,2.636849,2.960017,...,2.849721,2.924601,2.799268,2.884069,2.630465,2.530444,2.885181,2.881454,2.851810,2.711501
8,3.177504,3.191453,2.941501,3.095639,3.020481,3.061481,2.987817,2.997676,2.936294,3.170135,...,3.113950,3.156884,3.060505,3.178530,2.913853,2.836731,3.152729,3.157087,3.094390,2.996696


In [30]:
pd.DataFrame(test_matched_dist)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,3.142523,3.184640,2.889379,3.073822,2.990680,3.046007,2.956831,2.960849,2.887208,3.195108,...,3.093677,3.157711,3.012442,3.143373,2.874664,2.801604,3.128430,3.142101,3.084566,2.962097
1,3.147152,3.189600,2.901366,3.080598,2.995109,3.051160,2.962195,2.968305,2.898268,3.198869,...,3.100342,3.163365,3.021599,3.147124,2.880938,2.813657,3.135910,3.149373,3.091780,2.969781
2,3.302109,3.345925,3.064789,3.235460,3.153915,3.209332,3.133429,3.132395,3.065520,3.351130,...,3.259565,3.319770,3.186015,3.317648,3.050296,2.984167,3.294593,3.292218,3.244853,3.127144
3,3.151961,3.195378,2.906454,3.086972,3.001509,3.055820,2.967972,2.971295,2.903623,3.199811,...,3.103221,3.164592,3.025558,3.158560,2.888486,2.817041,3.143146,3.153918,3.088800,2.971958
4,3.506400,3.549550,3.254812,3.438030,3.349458,3.407113,3.335227,3.334938,3.257073,3.556708,...,3.467625,3.522757,3.366640,3.529232,3.228674,3.190341,3.500563,3.506006,3.439050,3.321094
5,3.194575,3.236366,2.950920,3.126836,3.044790,3.097101,3.012462,3.021262,2.947174,3.247366,...,3.147201,3.209973,3.068587,3.195875,2.935643,2.857905,3.184356,3.188840,3.134817,3.015248
6,3.174526,3.212261,2.924550,3.103529,3.015974,3.076607,2.984723,2.998521,2.923549,3.219058,...,3.124764,3.182587,3.044298,3.174679,2.907873,2.839744,3.162751,3.171059,3.109326,2.995577
7,3.134008,3.183787,2.897920,3.071558,2.984863,3.045419,2.959084,2.967675,2.893702,3.192925,...,3.094417,3.154241,3.022955,3.144083,2.878883,2.796800,3.130467,3.135985,3.084183,2.961085
8,3.155873,3.201666,2.903293,3.093170,3.003076,3.059330,2.983983,2.974983,2.903124,3.203925,...,3.110787,3.166885,3.024434,3.167228,2.883801,2.826320,3.148319,3.158717,3.094556,2.980145
